In [4]:
from torchsummary import summary
import torchvision.transforms as transforms
import torch.nn as nn
import torch

from src.nn.regression_dataset import RegressionDataset
from src.nn.create_data_loaders import create_data_loaders
from src.nn.cnn_regressor import CNNRegressor
from src.nn.training import training
from src.nn.plot_losses import plot_losses
import src.ctes.num_ctes as nctes

In [5]:
sample_size = nctes.LEN_SAMPLE

In [6]:
class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""
    def __call__(self, sample):
        if(torch.cuda.is_available()):
            return torch.FloatTensor(sample).cuda()
        else:
            return torch.FloatTensor(sample)

In [13]:
mrw_path = "../../data/MRW.npz"
transform = ToTensor()

data = RegressionDataset(mrw_path, transform, sample_size)

In [14]:
batch_size = 20
valid_size = 0.2
test_size = 0.2

train_loader, valid_loader, test_loader = create_data_loaders(batch_size, valid_size, test_size, data)

In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device ' + str(device))

Using device cuda


In [16]:
model = CNNRegressor(input_size=sample_size)
model.to(device=device)

summary(model, (1, sample_size))

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 16, 32768]           --
|    └─Conv1d: 2-1                       [-1, 16, 32768]           16
|    └─BatchNorm1d: 2-2                  [-1, 16, 32768]           32
|    └─ReLU: 2-3                         [-1, 16, 32768]           --
├─Sequential: 1-2                        [-1, 32, 32767]           --
|    └─Conv1d: 2-4                       [-1, 32, 32767]           1,024
|    └─BatchNorm1d: 2-5                  [-1, 32, 32767]           64
|    └─ReLU: 2-6                         [-1, 32, 32767]           --
├─AvgPool1d: 1-3                         [-1, 32, 16384]           --
├─Sequential: 1-4                        [-1, 64, 16381]           --
|    └─Conv1d: 2-7                       [-1, 64, 16381]           8,192
|    └─BatchNorm1d: 2-8                  [-1, 64, 16381]           128
|    └─ReLU: 2-9                         [-1, 64, 16381]           --
├─AvgPoo

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 16, 32768]           --
|    └─Conv1d: 2-1                       [-1, 16, 32768]           16
|    └─BatchNorm1d: 2-2                  [-1, 16, 32768]           32
|    └─ReLU: 2-3                         [-1, 16, 32768]           --
├─Sequential: 1-2                        [-1, 32, 32767]           --
|    └─Conv1d: 2-4                       [-1, 32, 32767]           1,024
|    └─BatchNorm1d: 2-5                  [-1, 32, 32767]           64
|    └─ReLU: 2-6                         [-1, 32, 32767]           --
├─AvgPool1d: 1-3                         [-1, 32, 16384]           --
├─Sequential: 1-4                        [-1, 64, 16381]           --
|    └─Conv1d: 2-7                       [-1, 64, 16381]           8,192
|    └─BatchNorm1d: 2-8                  [-1, 64, 16381]           128
|    └─ReLU: 2-9                         [-1, 64, 16381]           --
├─AvgPoo

In [17]:
criterion = nn.MSELoss()

In [18]:
params = model.parameters()
lr = 0.01

optimizer = torch.optim.SGD(params, lr)

In [19]:
n_epochs = 5

train_losses, valid_losses = training(n_epochs, train_loader, valid_loader, model, criterion, optimizer, device) 

RuntimeError: Given groups=1, weight of size [16, 1, 1], expected input[1, 20, 32768] to have 1 channels, but got 20 channels instead

In [ ]:
plot_losses([n_epochs, train_losses, valid_losses], ["Train", "Val"])

In [ ]:
model.load_state_dict(torch.load("model.pt", map_location=device))